In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
feature_id = 23
negative_rt_delta_tolerance = -0.25
positive_rt_delta_tolerance = 0.25
negative_scan_delta_tolerance = -4.0
positive_scan_delta_tolerance = 0.1
maximum_number_of_features = 1
maximum_number_of_peaks_per_feature = 100

In [4]:
DB_NAME = '/home/ubuntu/UPS2_allion/UPS2_allion-features-1-1097.sqlite'
CONV_DB_NAME = '/home/ubuntu/UPS2_allion/UPS2_allion.sqlite'

In [5]:
db_conn = sqlite3.connect(DB_NAME)
peak_correlation_df = pd.read_sql_query("select * from peak_correlation where feature_id=={} and rt_distance >= {} and rt_distance <= {} and scan_distance >= {} and scan_distance <= {} order by rt_distance ASC limit {}".format(feature_id, negative_rt_delta_tolerance, positive_rt_delta_tolerance, negative_scan_delta_tolerance, positive_scan_delta_tolerance, maximum_number_of_peaks_per_feature), db_conn)
db_conn.close()

In [13]:
peak_correlation_df["feature_id-ms2_peak_id"] = peak_correlation_df.feature_id.astype(str) + '-' + peak_correlation_df.ms2_peak_id.astype(str)

In [14]:
peak_correlation_df.head()

,feature_id,base_peak_id,ms1_scan_centroid,ms1_rt_centroid,ms2_peak_id,ms2_scan_centroid,ms2_rt_centroid,scan_distance,rt_distance,correlation,feature_id-ms2_peak_id
0,23,2,110.144302,1151.36315,835,111.295575,1151.612901,-1.151273,-0.249751,0.0,23-835
1,23,2,110.144302,1151.36315,4289,110.979236,1151.612824,-0.834934,-0.249674,0.0,23-4289
2,23,2,110.144302,1151.36315,14602,112.579343,1151.612801,-2.435041,-0.249651,0.0,23-14602
3,23,2,110.144302,1151.36315,7509,110.891836,1151.612357,-0.747534,-0.249207,0.0,23-7509
4,23,2,110.144302,1151.36315,9443,110.718558,1151.612078,-0.574256,-0.248928,0.0,23-9443


In [30]:
db_conn = sqlite3.connect(DB_NAME)
ms2_peaks_df = pd.read_sql_query("select feature_id,peak_id,centroid_mz,intensity from ms2_peaks where feature_id || '-' || peak_id in {}".format(tuple(peak_correlation_df["feature_id-ms2_peak_id"])), db_conn)
db_conn.close()

In [31]:
ms2_peaks_df.head()

,feature_id,peak_id,centroid_mz,intensity
0,23,835,627.372,73883
1,23,1436,599.275,32493
2,23,2047,609.997,29440
3,23,2481,649.970,26301
4,23,2806,633.224,19743


In [38]:
df = pd.merge(ms2_peaks_df, peak_correlation_df, left_on=['feature_id','peak_id'], right_on=['feature_id','ms2_peak_id'])

In [39]:
df.drop(['peak_id','correlation','feature_id-ms2_peak_id'], inplace=True, axis=1)

In [40]:
df.rename(columns={'intensity': 'ms2_peak_intensity', 'rt_distance': 'rt_delta', 'scan_distance': 'scan_delta', 'centroid_mz': 'ms2_peak_centroid_mz'}, inplace=True)

In [41]:
df.head()

,feature_id,ms2_peak_centroid_mz,ms2_peak_intensity,base_peak_id,ms1_scan_centroid,ms1_rt_centroid,ms2_peak_id,ms2_scan_centroid,ms2_rt_centroid,scan_delta,rt_delta
0,23,627.372,73883,2,110.144302,1151.36315,835,111.295575,1151.612901,-1.151273,-0.249751
1,23,599.275,32493,2,110.144302,1151.36315,1436,110.370664,1151.607068,-0.226362,-0.243918
2,23,609.997,29440,2,110.144302,1151.36315,2047,111.539878,1151.611239,-1.395576,-0.248089
3,23,649.970,26301,2,110.144302,1151.36315,2481,111.232957,1151.610221,-1.088655,-0.247071
4,23,633.224,19743,2,110.144302,1151.36315,2806,110.497240,1151.600965,-0.352937,-0.237815
